In [1]:
import time
from IPython.display import Audio
import logging
from concurrent.futures import ThreadPoolExecutor

from Freesound.data import *
from Freesound.utils import *
from Freesound.model import *
from Freesound.augmentations import *

import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

%reload_ext autoreload
%autoreload 2

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log", filemode="w+")

In [2]:
names, y_curated_np, y_noisy_np, train_curated, train_noisy, _, _ = get_tables()

train_audio = load_audio(1, False, 'Freesound/pickles/train_audio_44100.pkl')
train_audio.pop(534)
y_curated_np = np.delete(y_curated_np, 534, axis=0)
train_curated.drop(534, axis=0, inplace=True)
train_audio = np.array(train_audio)

mskf = MultilabelStratifiedKFold(n_splits=5, random_state=0)
train_index, test_index = next(mskf.split(y_curated_np, y_curated_np))
X_train, X_val = train_audio[train_index], train_audio[test_index]
y_train, y_val = y_curated_np[train_index], y_curated_np[test_index]

In [3]:
train_transform = transforms.Compose([
    #RandomParameter(VTLP, [[0.8, 1.2]], p=0.5),
    #MinMaxChunkScaler(),
    #Normalize(),
    RandomParameter(RandomNoise, [[0.01, 0.1]]),
    #RandomParameter(Shift, [[2000, 32000]]),
    RandomParameter(TimeStretch, [[0.75, 1.3]]),
    #RandomParameter(PitchShift, [[-8, 8]]),
    #RandomParameter(Distortion, [[-1, -0.3], [.3, 1.]]),
    ToMellSpec(n_mels=256),
    #GetMFCC(),
    #PadOrClip(300),
    #Normalize(),
    #ToTensor(),
    #transforms.ToTensor(),
])

In [4]:
mskf = MultilabelStratifiedKFold(n_splits=5, random_state=0)

train_index, test_index = next(mskf.split(y_curated_np, y_curated_np))
X_train, X_val = train_audio[train_index], train_audio[test_index]
y_train, y_val = y_curated_np[train_index], y_curated_np[test_index]

In [5]:
##train
train_dataset = Dataset_Train(X_train, y_train, train_transform, preload=True)
valid_dataset = Dataset_Train(X_val, y_val, train_transform, preload=True)

In [6]:
def precalculate_and_save_one_epoch(folder, run, dataset):
    for sample in tqdm(range(len(dataset))):
        k = dataset[sample][0]
        name = f"{run}_{sample}"
        path = os.path.join(folder, name)
        np.save(path, k)

In [7]:
def save_sample(sample, dataset, run, folder):
    k = dataset[sample][0]
    name = f"{run}_{sample}"
    path = os.path.join(folder, name)
    np.save(path, k)

In [8]:
def precalculate_and_save_one_epoch_parallel(run, folder, dataset):
    before = time.time()
    with ThreadPoolExecutor(max_workers=16) as executor:
        for sample in range(len(dataset)):
            future = executor.submit(save_sample, sample, dataset, run, folder)
    
    print(time.time() - before)

In [11]:
!rm -rf ../data/raw_mels_augs_RT
!rm -rf ../data/raw_mels_augs_RT_val

In [12]:
!mkdir ../data/raw_mels_augs_RT
!mkdir ../data/raw_mels_augs_RT_val

In [ ]:
for num in range(0, 15):
    precalculate_and_save_one_epoch_parallel(num, "../data/raw_mels_augs_RT", train_dataset)
    precalculate_and_save_one_epoch_parallel(num, "../data/raw_mels_augs_RT_val", valid_dataset)
    print(num)

317.66523814201355
89.53032302856445
0
307.2045338153839
85.75043392181396
1
320.8328206539154
78.1231620311737
2
320.2545087337494
77.91554188728333
3
